In [58]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math
from keras.models import Sequential
from keras.layers import Dense,Input
import keras

In [59]:
look_back = 1

def create_dataset(dataset):
    dataX, dataY = [], []
    for i in range(len(dataset) - look_back  - 1):
        x = dataset[i: i + look_back, 0]
        dataX.append(x)
        y = dataset[i + look_back,0]
        dataY.append(y)
    return np.array(dataX),np.array(dataY)

def build_model():
    model = Sequential()
    model.add(Input(shape=(look_back,)))
    model.add(Dense(units=8,activation='relu'))
    model.add(Dense(units=1))
    model.compile(loss='mean_squared_error',optimizer='adam')
    return model

In [60]:
seed = 7
batch_size = 2
epochs = 200
filename = 'data/floating_polulation.csv'
footer = 1

# 设置随机种子
np.random.seed(seed)

# 导入数据
data = pd.read_csv(filename,usecols=[1],engine='python',skipfooter=footer)
dataset = data.values.astype('float32')
train_size = int(len(dataset)* 0.67)
validation_size = len(dataset) - train_size
train,validation = dataset[0:train_size,:],dataset[train_size:len(dataset),:]

# 创建dataset，让数据产生相关性
X_train,Y_trian = create_dataset(train)
X_validation,Y_validation = create_dataset(validation)

# 训练模型
model = build_model()
model.fit(X_train,Y_trian, epochs=epochs, batch_size=batch_size, verbose=2)

Epoch 1/200
47/47 - 1s - 12ms/step - loss: 5562964992.0000
Epoch 2/200
47/47 - 0s - 1ms/step - loss: 4885542400.0000
Epoch 3/200
47/47 - 0s - 796us/step - loss: 4269844480.0000
Epoch 4/200
47/47 - 0s - 855us/step - loss: 3680642048.0000
Epoch 5/200
47/47 - 0s - 854us/step - loss: 3109935104.0000
Epoch 6/200
47/47 - 0s - 845us/step - loss: 2543417344.0000
Epoch 7/200
47/47 - 0s - 840us/step - loss: 1995931776.0000
Epoch 8/200
47/47 - 0s - 822us/step - loss: 1493015552.0000
Epoch 9/200
47/47 - 0s - 859us/step - loss: 1054853888.0000
Epoch 10/200
47/47 - 0s - 847us/step - loss: 701154432.0000
Epoch 11/200
47/47 - 0s - 825us/step - loss: 436356864.0000
Epoch 12/200
47/47 - 0s - 673us/step - loss: 251788080.0000
Epoch 13/200
47/47 - 0s - 852us/step - loss: 136789072.0000
Epoch 14/200
47/47 - 0s - 815us/step - loss: 70180464.0000
Epoch 15/200
47/47 - 0s - 848us/step - loss: 35719784.0000
Epoch 16/200
47/47 - 0s - 798us/step - loss: 19424726.0000
Epoch 17/200
47/47 - 0s - 897us/step - loss: 1

In [77]:
keras.saving.save_model(model, 'model/floating_model.keras')

### 评估模型

In [62]:
train_score = model.evaluate(X_train,Y_trian,verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)'%(train_score,math.sqrt(train_score)))
validation_score = model.evaluate(X_validation,Y_validation,verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (validation_score, math.sqrt(validation_score)))

Train Score: 7593358.50 MSE (2755.60 RMSE)
Train Score: 16031425.00 MSE (4003.93 RMSE)


### 构建通过训练数据集进行预测的图表数据

In [63]:
# 利用图表查看预测趋势
predict_train = model.predict(X_train)
predict_validation = model.predict(X_validation)

predict_train_plot = np.empty_like(dataset)
predict_train_plot[:,:] = np.nan
predict_train_plot[look_back:len(predict_train) + look_back,:] = predict_train

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


### 构建通过评估数据集进行预测的图表数据

In [64]:
predict_validation_plot = np.empty_like(dataset)
predict_validation_plot[:,:] = np.nan
predict_validation_plot[len(predict_train) + look_back * 2 + 1:len(dataset) - 1,:] = predict_validation

### R方值评估

In [65]:
from sklearn.metrics import r2_score

r2_valid = r2_score(Y_validation, predict_validation)
r2_valid 

0.8210322856903076

In [74]:
X_train.shape
# train.shape

(93, 1)

### 单条预测

In [76]:
pred = model.predict(np.array([3568]))
int(pred[0][0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


3584